In [1]:
import pandas as pd
import numpy as np
from allocations import Allocations
from metrics import Metrics

In [2]:
df = pd.read_csv("predictions/obermeyer/bootstrap_regression.csv")
data = pd.read_csv("data/obermeyer/obermeyer_data_cleaned.csv")

In [3]:
TEST_SIZE = 100
SELECTION_RATE = 0.25
QUALIFICATION_RATE = 0.50
RASHOMON_EPSILON = 0.01
ITERATIONS_PER_SPLIT = 20

a = Allocations(df, TEST_SIZE, SELECTION_RATE, QUALIFICATION_RATE, RASHOMON_EPSILON, ITERATIONS_PER_SPLIT)

In [4]:
allocations = a.get_allocations()

In [5]:
m = Metrics(allocations, data, QUALIFICATION_RATE)

In [6]:
m.k_prime()

(np.float64(20.74), np.float64(2.081441807978306))

In [7]:
m.n_prime()

(np.float64(50.63), np.float64(5.070808614018084))

In [8]:
m.count_rashomon_allocations()

(np.float64(6.69), np.float64(6.167163043085531))

In [9]:
m.count_rashomon_models()

(np.float64(15.3), np.float64(17.916193792209327))

In [10]:
m.count_rashomon_models_per_allocation()

(np.float64(2.2104591208391806), np.float64(2.486918868643382))

In [11]:
m.systemic_exclusion()

(np.float64(71.87), np.float64(2.270044052435987))

In [12]:
m.selections_by_qualification()

(np.float64(2.7398267963257923),
 np.float64(3.1126838136192494),
 np.float64(0.5552467759452719),
 np.float64(1.6872348806687985))

In [13]:
m.selections_by_qualification()

(np.float64(2.7398267963257923),
 np.float64(3.1126838136192494),
 np.float64(0.5552467759452719),
 np.float64(1.6872348806687985))

In [14]:
m.minority_selection_rate("race", 1)

(np.float64(0.1549329779981804),
 np.float64(0.06900361103611467),
 np.float64(0.1668))

In [15]:
m.best_minority_selection_rate("race", 1)

(np.float64(0.36519999999999997), np.float64(0.11657169467756742))

In [16]:
m.feature_ratio_by_group("race", 1, 0, "gagne_sum_t")

/Users/shomik/Documents/MIT/Rashomon/rashomon/metrics.py:165: RuntimeWarning: Mean of empty slice
  metric_flattened = [np.nanmean(inner) for inner in metric]


(np.float64(1.457984399204924),
 np.float64(0.7013155766146071),
 np.float64(1.3506357012638748))

In [17]:
m.best_feature_ratio_by_group("race", 1, 0, "gagne_sum_t")

(np.float64(1.1280311933220735), np.float64(0.49519348149135334))